# google-covid-19-mobility-data-process-usa

In this notebook I will refine the process from `google-covid-19-mobility-data-process-v1` in order to get the highest resolution data possible for a map of the USA.

In [1]:
import pandas as pd
import requests
import simplejson as json
import numpy as np

---

## Load a reduced CSV containing just the United States entries

In [2]:
usaDf = pd.read_csv("./output-data/usa.csv", low_memory=False)

In [3]:
usaDf.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,US,United States,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
1,US,United States,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0
2,US,United States,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-17,6.0,0.0,28.0,-9.0,-24.0,5.0
3,US,United States,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-18,0.0,-1.0,6.0,1.0,0.0,1.0
4,US,United States,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-19,2.0,0.0,8.0,1.0,1.0,0.0


---

**`sub_reigon_1` is states**

In [9]:
usaDf["sub_region_1"].unique()

array([nan, 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

---

**`sub_region_2` is counties - of which there are 1,719**

In [10]:
usaDf["sub_region_2"].unique()

array([nan, 'Autauga County', 'Baldwin County', ..., 'Uinta County',
       'Washakie County', 'Weston County'], dtype=object)

In [11]:
len(usaDf["sub_region_2"].unique())

1719

---

**`metro_area` is not used**

In [13]:
usaDf["metro_area"].unique()

array([nan])

---

**`iso_3166_2_code` is used and relates to States.**

In [14]:
usaDf["iso_3166_2_code"].unique()

array([nan, 'US-AL', 'US-AK', 'US-AZ', 'US-AR', 'US-CA', 'US-CO', 'US-CT',
       'US-DE', 'US-DC', 'US-FL', 'US-GA', 'US-HI', 'US-ID', 'US-IL',
       'US-IN', 'US-IA', 'US-KS', 'US-KY', 'US-LA', 'US-ME', 'US-MD',
       'US-MA', 'US-MI', 'US-MN', 'US-MS', 'US-MO', 'US-MT', 'US-NE',
       'US-NV', 'US-NH', 'US-NJ', 'US-NM', 'US-NY', 'US-NC', 'US-ND',
       'US-OH', 'US-OK', 'US-OR', 'US-PA', 'US-RI', 'US-SC', 'US-SD',
       'US-TN', 'US-TX', 'US-UT', 'US-VT', 'US-VA', 'US-WA', 'US-WV',
       'US-WI', 'US-WY'], dtype=object)

---

**`census_fips_code` is present and could be used as an alternative to `place_id`**

In [15]:
usaDf["census_fips_code"].unique()

array([   nan,  1001.,  1003., ..., 56041., 56043., 56045.])

---

## Create a new data frame containing just the rows with `sub_region_2`

In [4]:
len(usaDf["place_id"].unique())

2889

In [5]:
usaSubRegion2Df = usaDf[usaDf["sub_region_2"].notnull()]

In [6]:
len(usaSubRegion2Df["place_id"].unique())

2837

We now have 2,837 rows instead of 2,889. Not a great reduction...

---

## Get the lat long coordinates for each unique `place_id`

In [29]:
uniquePlaceIdsDf = usaSubRegion2Df[["place_id"]].drop_duplicates()

Access the google maps api to get coordinates for each `place_id`

In [30]:
with open('./secrets/googleapikey.txt', 'r') as f:
    key = f.read()

In [31]:
def get_lat_long(place_id):
    try:
        API_KEY = key.rstrip("\n")
        url = "https://maps.googleapis.com/maps/api/place/details/json?place_id=" + str(place_id) + "&key=" + str(API_KEY) + "&fields=geometry"
        
        response = (requests.get(url).text)
        response_json = json.loads(response)
        
        if "result" in response_json:
            result = response_json["result"]
            if "geometry" in result:
                geometry = result["geometry"]
                if "location" in geometry:
                    location = geometry["location"]
                    return location["lat"], location["lng"]
                else:
                    return None, None
            else:
                return None, None
        else:
            return None, None
    
    except Exception as e:
        raise e

In [32]:
uniquePlaceIdsDf.loc[:, "lat"], uniquePlaceIdsDf.loc[:, "lng"] = zip(*uniquePlaceIdsDf['place_id'].map(get_lat_long))

In [33]:
uniquePlaceIdsDf.head()

,place_id,lat,lng
924,ChIJg9z7ewWPjogRA_8QrB0va7o,32.579182,-86.499655
1380,ChIJMRERvxBnmogR0K_9O08dxDw,30.601074,-87.776333
1842,ChIJtUJfgR1tjYgRAjFO7yq2nuI,31.817290,-85.354965
2279,ChIJEdvIA8szj4gRU_XpIL_1cSw,32.956280,-87.142289
2716,ChIJdcUJhYeXiYgRERpotPlZFfs,34.014515,-86.499655


I'll save these coordinates as a CSV file for later use.

In [34]:
uniquePlaceIdsDf.to_csv("./output-data/usa-sub-region-2-lat-lng.csv", index=False)

In [35]:
len(uniquePlaceIdsDf)

2837

---

## Merge the coordinates with the original `sub_region_2` data frame

In [7]:
uniquePlaceIdsDf = pd.read_csv("./output-data/usa-sub-region-2-lat-lng.csv", low_memory=False)

In [8]:
usaSubRegion2MergeDf = pd.merge(usaSubRegion2Df, uniquePlaceIdsDf, on='place_id', how='outer')

In [9]:
usaSubRegion2MergeDf.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,lat,lng
0,US,United States,Alabama,Autauga County,NaN,NaN,1001.0,ChIJg9z7ewWPjogRA_8QrB0va7o,2020-02-15,5.0,7.0,NaN,NaN,-4.0,NaN,32.579182,-86.499655
1,US,United States,Alabama,Autauga County,NaN,NaN,1001.0,ChIJg9z7ewWPjogRA_8QrB0va7o,2020-02-16,0.0,1.0,-23.0,NaN,-4.0,NaN,32.579182,-86.499655
2,US,United States,Alabama,Autauga County,NaN,NaN,1001.0,ChIJg9z7ewWPjogRA_8QrB0va7o,2020-02-17,8.0,0.0,NaN,NaN,-27.0,5.0,32.579182,-86.499655
3,US,United States,Alabama,Autauga County,NaN,NaN,1001.0,ChIJg9z7ewWPjogRA_8QrB0va7o,2020-02-18,-2.0,0.0,NaN,NaN,2.0,0.0,32.579182,-86.499655
4,US,United States,Alabama,Autauga County,NaN,NaN,1001.0,ChIJg9z7ewWPjogRA_8QrB0va7o,2020-02-19,-2.0,0.0,NaN,NaN,2.0,0.0,32.579182,-86.499655


---

## Calculate 7 day rolling averages for each location

In [10]:
def add_rolling_average(df):
    df.loc[:, "parks-average"] = df["parks_percent_change_from_baseline"].rolling(window=7, center=True).mean()
    return df

In [11]:
usaSubRegion2AverageDf = usaSubRegion2MergeDf.groupby("place_id").apply(add_rolling_average)

In [35]:
usaSubRegion2AverageDf["parks-average"].dropna()[0:10]

459    26.428571
460    26.571429
461    41.428571
462    38.714286
463    40.571429
464    40.142857
465    47.142857
466    48.142857
467    55.714286
468    50.142857
Name: parks-average, dtype: float64

---

## Round the average figures to 1 decimal place for a smaller final file size

In [36]:
usaSubRegion2RoundedDf = usaSubRegion2AverageDf.round({
    'parks-average': 1,
})

In [37]:
usaSubRegion2RoundedDf["parks-average"].dropna()[0:10]

459    26.4
460    26.6
461    41.4
462    38.7
463    40.6
464    40.1
465    47.1
466    48.1
467    55.7
468    50.1
Name: parks-average, dtype: float64

---

## Remove any NaN `place_id`s

In [38]:
usaSubRegion2NotNaDf = usaSubRegion2RoundedDf[usaSubRegion2RoundedDf["place_id"].notna(
)]

---

## Convert the data into a python dictionary so it can be exported as json

In [39]:
dates = ["2020-02-15","2020-02-16","2020-02-17","2020-02-18","2020-02-19","2020-02-20","2020-02-21","2020-02-22","2020-02-23","2020-02-24","2020-02-25","2020-02-26","2020-02-27","2020-02-28","2020-02-29","2020-03-01","2020-03-02","2020-03-03","2020-03-04","2020-03-05","2020-03-06","2020-03-07","2020-03-08","2020-03-09","2020-03-10","2020-03-11","2020-03-12","2020-03-13","2020-03-14","2020-03-15","2020-03-16","2020-03-17","2020-03-18","2020-03-19","2020-03-20","2020-03-21","2020-03-22","2020-03-23","2020-03-24","2020-03-25","2020-03-26","2020-03-27","2020-03-28","2020-03-29","2020-03-30","2020-03-31","2020-04-01","2020-04-02","2020-04-03","2020-04-04","2020-04-05","2020-04-06","2020-04-07","2020-04-08","2020-04-09","2020-04-10","2020-04-11","2020-04-12","2020-04-13","2020-04-14","2020-04-15","2020-04-16","2020-04-17","2020-04-18","2020-04-19","2020-04-20","2020-04-21","2020-04-22","2020-04-23","2020-04-24","2020-04-25","2020-04-26","2020-04-27","2020-04-28","2020-04-29","2020-04-30","2020-05-01","2020-05-02","2020-05-03","2020-05-04","2020-05-05","2020-05-06","2020-05-07","2020-05-08","2020-05-09","2020-05-10","2020-05-11","2020-05-12","2020-05-13","2020-05-14","2020-05-15","2020-05-16","2020-05-17","2020-05-18","2020-05-19","2020-05-20","2020-05-21","2020-05-22","2020-05-23","2020-05-24","2020-05-25","2020-05-26","2020-05-27","2020-05-28","2020-05-29","2020-05-30","2020-05-31","2020-06-01","2020-06-02","2020-06-03","2020-06-04","2020-06-05","2020-06-06","2020-06-07","2020-06-08","2020-06-09","2020-06-10","2020-06-11","2020-06-12","2020-06-13","2020-06-14","2020-06-15","2020-06-16","2020-06-17","2020-06-18","2020-06-19","2020-06-20","2020-06-21","2020-06-22","2020-06-23","2020-06-24","2020-06-25","2020-06-26","2020-06-27","2020-06-28","2020-06-29","2020-06-30","2020-07-01","2020-07-02","2020-07-03","2020-07-04","2020-07-05","2020-07-06","2020-07-07","2020-07-08","2020-07-09","2020-07-10","2020-07-11","2020-07-12","2020-07-13","2020-07-14","2020-07-15","2020-07-16","2020-07-17","2020-07-18","2020-07-19","2020-07-20","2020-07-21","2020-07-22","2020-07-23","2020-07-24","2020-07-25","2020-07-26","2020-07-27","2020-07-28","2020-07-29","2020-07-30","2020-07-31","2020-08-01","2020-08-02","2020-08-03","2020-08-04","2020-08-05","2020-08-06","2020-08-07","2020-08-08","2020-08-09","2020-08-10","2020-08-11","2020-08-12","2020-08-13","2020-08-14","2020-08-15","2020-08-16","2020-08-17","2020-08-18","2020-08-19","2020-08-20","2020-08-21","2020-08-22","2020-08-23","2020-08-24","2020-08-25","2020-08-26","2020-08-27","2020-08-28","2020-08-29","2020-08-30","2020-08-31","2020-09-01","2020-09-02","2020-09-03","2020-09-04","2020-09-05","2020-09-06","2020-09-07","2020-09-08","2020-09-09","2020-09-10","2020-09-11","2020-09-12","2020-09-13","2020-09-14","2020-09-15","2020-09-16","2020-09-17","2020-09-18","2020-09-19","2020-09-20","2020-09-21","2020-09-22","2020-09-23","2020-09-24","2020-09-25","2020-09-26","2020-09-27","2020-09-28","2020-09-29","2020-09-30","2020-10-01","2020-10-02","2020-10-03","2020-10-04","2020-10-05","2020-10-06","2020-10-07","2020-10-08","2020-10-09","2020-10-10","2020-10-11","2020-10-12","2020-10-13","2020-10-14","2020-10-15","2020-10-16","2020-10-17","2020-10-18","2020-10-19","2020-10-20","2020-10-21","2020-10-22","2020-10-23","2020-10-24","2020-10-25","2020-10-26","2020-10-27","2020-10-28","2020-10-29","2020-10-30","2020-10-31","2020-11-01","2020-11-02","2020-11-03","2020-11-04","2020-11-05","2020-11-06","2020-11-07","2020-11-08","2020-11-09","2020-11-10","2020-11-11","2020-11-12","2020-11-13","2020-11-14","2020-11-15","2020-11-16","2020-11-17","2020-11-18","2020-11-19","2020-11-20","2020-11-21","2020-11-22","2020-11-23","2020-11-24","2020-11-25","2020-11-26","2020-11-27","2020-11-28","2020-11-29","2020-11-30","2020-12-01","2020-12-02","2020-12-03","2020-12-04","2020-12-05","2020-12-06","2020-12-07","2020-12-08","2020-12-09","2020-12-10","2020-12-11","2020-12-12","2020-12-13","2020-12-14","2020-12-15","2020-12-16","2020-12-17","2020-12-18","2020-12-19","2020-12-20","2020-12-21","2020-12-22","2020-12-23","2020-12-24","2020-12-25","2020-12-26","2020-12-27","2020-12-28","2020-12-29","2020-12-30","2020-12-31","2021-01-01","2021-01-02","2021-01-03","2021-01-04","2021-01-05","2021-01-06","2021-01-07","2021-01-08","2021-01-09","2021-01-10","2021-01-11","2021-01-12","2021-01-13","2021-01-14","2021-01-15","2021-01-16","2021-01-17","2021-01-18","2021-01-19","2021-01-20","2021-01-21","2021-01-22","2021-01-23","2021-01-24","2021-01-25","2021-01-26","2021-01-27","2021-01-28","2021-01-29","2021-01-30","2021-01-31","2021-02-01","2021-02-02","2021-02-03","2021-02-04","2021-02-05","2021-02-06","2021-02-07","2021-02-08","2021-02-09","2021-02-10","2021-02-11","2021-02-12","2021-02-13","2021-02-14","2021-02-15","2021-02-16","2021-02-17","2021-02-18","2021-02-19","2021-02-20","2021-02-21","2021-02-22","2021-02-23","2021-02-24","2021-02-25","2021-02-26","2021-02-27","2021-02-28","2021-03-01","2021-03-02","2021-03-03","2021-03-04","2021-03-05","2021-03-06","2021-03-07","2021-03-08","2021-03-09","2021-03-10","2021-03-11","2021-03-12","2021-03-13","2021-03-14","2021-03-15","2021-03-16","2021-03-17","2021-03-18","2021-03-19","2021-03-20","2021-03-21","2021-03-22","2021-03-23","2021-03-24","2021-03-25","2021-03-26","2021-03-27","2021-03-28","2021-03-29","2021-03-30","2021-03-31","2021-04-01","2021-04-02","2021-04-03","2021-04-04","2021-04-05","2021-04-06","2021-04-07","2021-04-08","2021-04-09","2021-04-10","2021-04-11","2021-04-12","2021-04-13","2021-04-14","2021-04-15","2021-04-16","2021-04-17","2021-04-18","2021-04-19","2021-04-20","2021-04-21","2021-04-22","2021-04-23","2021-04-24","2021-04-25","2021-04-26","2021-04-27","2021-04-28","2021-04-29","2021-04-30","2021-05-01","2021-05-02","2021-05-03","2021-05-04","2021-05-05","2021-05-06","2021-05-07","2021-05-08","2021-05-09","2021-05-10","2021-05-11","2021-05-12","2021-05-13","2021-05-14","2021-05-15","2021-05-16","2021-05-17","2021-05-18","2021-05-19","2021-05-20","2021-05-21"]

In [40]:
def create_list_for_json(df):
    outputList = []
    listOfPlaceIds = df["place_id"].drop_duplicates().to_list()
    groupByPlaceId = df.groupby("place_id")
    
    for place_id in listOfPlaceIds:
        thisDf = groupByPlaceId.get_group(place_id)
        testList = thisDf["parks-average"].to_list()
        
        # Some of the parks columns contain all NaNs, we'll skip these
        if np.isnan(testList).all():
            continue
        
        myDict = {}
        myDict['lng'] = thisDf.iloc[0]["lng"]
        myDict["lat"] =  thisDf.iloc[0]["lat"]
        
        parksList = []
        
        dateIndexDf = thisDf.set_index("date")
        
        for date in dates:
            thisDate = np.nan
            try:
                thisDate = dateIndexDf.loc[date]["parks-average"]
            except:
                thisDate = np.nan
            
            parksList.append(thisDate)
        
        myDict["parks"] = parksList

        outputList.append(myDict)
        
    return outputList

In [41]:
usaSubRegion2List = create_list_for_json(usaSubRegion2NotNaDf)

In [47]:
len(usaSubRegion2List)

666

In [48]:
with open("./public/data/usa-parks.json", "w") as outfile: 
    json.dump(usaSubRegion2List, outfile, ignore_nan=True)